In [1]:
import os.path as p
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import pickle
import seaborn as sns


from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from torch.utils.data.dataloader import DataLoader
from definitions import *
from model_helper_functions import *
from dataset_helper_functions import *
from BiLSTM import BiLSTM
from DebatesDataset import DebatesDataset


In [2]:
try:
    IS_MASTER
except: 
    IS_MASTER = False

In [3]:
if not IS_MASTER:
    data, features = {}, {}
    emb_model = 'all-mpnet-base-v2'#'multi-qa-mpnet-base-dot-v1'#'all-MiniLM-L6-v2'
    
    dev_path = p.join(PROC_DATA_DIR_PATH, 'dev')
    features_path = p.join(PROC_DATA_DIR_PATH, 'features')

    data_paths = {
        'dev': [
            p.join(dev_path, 'dev.tsv'),
            p.join(dev_path, 'dev_spacy.pkl'),
            p.join(features_path, 'dev_stylometric_features.pkl'),
        ],
        'test': [
            p.join(POLIT_DATA_DIR_PATH, 'test', 'test_combined.tsv'),
            p.join(PROC_DATA_DIR_PATH, 'test', 'test_spacy.pkl'),
            p.join(features_path, 'test_stylometric_features.pkl'),

        ],
        'train': [
            p.join(POLIT_DATA_DIR_PATH, 'train', 'train_combined.tsv'),
            p.join(PROC_DATA_DIR_PATH, 'train', 'train_spacy.pkl'),
            p.join(features_path, 'train_stylometric_features.pkl')
        ],
        # 'val': [
        #     p.join(POLIT_DATA_DIR_PATH, 'val', 'val_combined.tsv'),
        #     p.join(PROC_DATA_DIR_PATH, 'val', 'val_spacy.pkl'),
        #     p.join(embeddings_path, f'val_sent_emb_{emb_model}.pkl'),
        #     p.join(features_path, 'val_stylometric_features.pkl'),
        # ],
    }

    for dtype, dpaths in data_paths.items():
        try:
            data[dtype] = pd.read_pickle(dpaths[1])
            features[dtype] = pd.read_pickle(dpaths[2])
        except Exception as e:
            print(e.args)
            exit()

Wrap data in `DebatesDataset` class so that it can be passed to `DataLoader` class.

In [4]:
dd_train = DebatesDataset(data=data['train'])
dd_test = DebatesDataset(data=data['test'])

loader_train = DataLoader(dd_train, batch_size=32, shuffle=True)
loader_test = DataLoader(dd_test, batch_size=32, shuffle=True)

Initialize model, optimizer and criterion.

In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = BiLSTM(device=device)
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.BCELoss()
num_epochs = 5
eval_every = len(loader_train) // 2
best_valid_loss = float("Inf")

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Initialize running values.

In [6]:
running_loss = 0.0
valid_running_loss = 0.0
global_step = 0
train_loss_list = []
valid_loss_list = []
global_steps_list = []

Training process.

In [8]:
model.train()

for epoch in range(num_epochs):
    for ids, sentences, labels  in loader_train:
        pred = model(sentences)
        print(pred.cpu())
        
    break

RuntimeError: The expanded size of the tensor (582) must match the existing size (512) at non-singleton dimension 1.  Target sizes: [1, 582].  Tensor sizes: [1, 512]